# Projeto Final - Bootcamp CDIA
Aluno: Paulo Roberto Carregosa Luiz

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
import numpy as np


In [ ]:
# Carregar os dados
df_train = pd.read_excel('bootcamp_train.xlsx')
df_test = pd.read_excel('bootcamp_test.xlsx')

In [ ]:
# Distribuição da área
sns.histplot(df_train['area_pixels'], bins=50, kde=True)
plt.title("Distribuição da Área dos Defeitos")
plt.xlabel("Área (pixels)")
plt.ylabel("Frequência")
plt.show()

In [ ]:
# Matriz de correlação
corr = df_train.select_dtypes(include='number').corr()
plt.figure(figsize=(12, 10))
sns.heatmap(corr, cmap="coolwarm", center=0)
plt.title("Matriz de Correlação")
plt.show()

In [ ]:
# Selecionar features e target
target_cols = ['falha_1', 'falha_2', 'falha_3', 'falha_4', 'falha_5', 'falha_6', 'falha_outros']
X = df_train.drop(columns=['id'] + target_cols)
y = df_train[target_cols]

# Corrigir colunas categóricas
X = X.replace({'nÃ£o': 0, 'NÃ£o': 0, 'Sim': 1, 'True': 1, 'False': 0})

# Preencher valores faltantes
X = X.fillna(0)

# Escalonar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Treinar modelo Random Forest
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Avaliação
y_pred = model.predict(X_val)
print("F1 Score (macro):", f1_score(y_val, y_pred, average='macro'))
print(classification_report(y_val, y_pred))

In [ ]:
# Aplicar modelo ao conjunto de teste
X_test = df_test.drop(columns=['id'])
X_test = X_test.replace({'nÃ£o': 0, 'NÃ£o': 0, 'Sim': 1, 'True': 1, 'False': 0})
X_test = X_test.fillna(0)
X_test_scaled = scaler.transform(X_test)

# Gerar predições
y_test_pred = model.predict(X_test_scaled)
y_test_pred_df = pd.DataFrame(y_test_pred, columns=target_cols)
y_test_pred_df.insert(0, 'id', df_test['id'])
y_test_pred_df.head()

In [ ]:
# Salvar modelo e scaler para uso no dashboard
import joblib

joblib.dump(model, 'modelo_random_forest.pkl')
joblib.dump(scaler, 'scaler.pkl')